In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import importlib
from src import html_requests
from src import parse_html
from src.pfr_scraper import PfrScraper

In [9]:
# If making changes to python modules, need to reimport them each time
importlib.reload(html_requests)
importlib.reload(parse_html)

<module 'src.parse_html' from '/Users/Jared/dev_work/projects/sports-reference-scrapers/src/parse_html.py'>

In [7]:
# Get Season Summaries for Seasons 1970-->2021:
save_path = '/Users/Jared/OneDrive - Sportsbook Science/data/raw/nfl/pfr_season_summaries'
driver_path = '/Users/Jared/dev_work/chromedriver'
html_requests.get_season_summaries(seasons=range(1970,2022),
                                   folder_path=save_path,
                                   chromedriver_path=driver_path)

Getting 1970 Season Summary
Getting 1971 Season Summary


In [135]:
# Get needed week summary info for seasons 1970->2021
years = range(1970,2022)
weeks_lkp = {}
for season in years:
    weeks_lkp[season] = []
    with open("{}/{}_summary.html".format(season_save_location, season)) as f:
        season_page = f.read()
    soup = BeautifulSoup(season_page, 'html.parser')
    week_elements = soup.find(id='div_week_games').find_all('a')
    week_num = 0
    for week_element in week_elements:
        week_num += 1
        weeks_lkp[season].append([week_num, week_element['href'], week_element.get_text()])

In [75]:
# Get Web Pages for all weeks - REQUESTS HAPPENING
base_save_location = '/Users/Jared/OneDrive - Sportsbook Science/data/raw/nfl'
week_summary_location = base_save_location+'/pfr_week_summaries'
for season_key in weeks_lkp.keys():
    for week_summary_element in weeks_lkp[season_key]:
        week_summary_response = requests.get(base_url+week_summary_element[1])
        with open('{}/{}_week{}_summary.html'.format(week_summary_location, season_key, week_summary_element[0]), 'w+') as f:
            f.write(week_summary_response.text)
        time.sleep(3)

In [139]:
# Parse out game links for each week of each season
for season_key in weeks_lkp.keys():
    indx = 0
    for week_summary_element in weeks_lkp[season_key]:
        with open("{}/{}_week{}_summary.html".format(week_summary_location, season_key, week_summary_element[0])) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        game_link_elements = soup.find_all('td', class_='right gamelink')
        game_links = []
        for game_link_element in game_link_elements:
            game_links.append(game_link_element.find('a')['href'])
        weeks_lkp[season_key][indx].append(game_links)
        indx += 1

# Saving the lookup dictionary so it isn't lost going forward (between sessions)
import pickle
with open ('tmp_vars/season_week_lkp_dict.pkl', 'wb') as f:
    pickle.dump(weeks_lkp, f)

In [181]:
# Send request to take a look at the box score/single game info - see if javascript reader is needed
year_to_test = 2021
week_to_test = 5
game = 3
game_save_location = base_save_location+'/pfr_game_summaries'
tester_games = weeks_lkp[year_to_test][week_to_test-1][3]
tester_game = tester_games[game]
print(tester_game)
game_id = tester_game[tester_game.find(tester_game.split('/')[2]):tester_game.find('.htm')]
tester_game_response = requests.get(base_url+tester_game)
with open(f'{game_save_location}/{game_id}.html', 'w+') as f:
    f.write(tester_game_response.text)

/boxscores/202110100cin.htm


In [182]:
# Take a look at game info with BS
year_to_test = 2021
week_to_test = 5
game = 3
tester_games = weeks_lkp[year_to_test][week_to_test-1][3]
tester_game = tester_games[game]
game_id = tester_game[tester_game.find(tester_game.split('/')[2]):tester_game.find('.htm')]

with open (f'{game_save_location}/{game_id}.html') as f:
    tester_html = f.read()
soup = BeautifulSoup(tester_html, 'html.parser')

# Game Summary Info

# Get Linescore
linescore = str(soup.find('table',class_='linescore nohover stats_table no_freeze'))
linescore_df = pd.read_html(linescore)[0]
entry = {
    'away_team': linescore_df['Unnamed: 1'][0],
    'away_team_score_q1': linescore_df['1'][0],
    'away_team_score_q2': linescore_df['2'][0],
    'away_team_score_q3': linescore_df['3'][0],
    'away_team_score_q4': linescore_df['4'][0],
    'away_team_score_final': linescore_df['Final'][0],
    'home_team': linescore_df['Unnamed: 1'][1],
    'home_team_score_q1': linescore_df['1'][1],
    'home_team_score_q2': linescore_df['2'][1],
    'home_team_score_q3': linescore_df['3'][1],
    'home_team_score_q4': linescore_df['4'][1],
    'home_team_score_final': linescore_df['Final'][1]
}
# Check Overtime
if 'OT' in linescore_df.columns:
    entry['away_team_score_overtime'] = linescore_df['OT'][0]
    entry['home_team_score_overtime'] = linescore_df['OT'][1]
else:
    entry['away_team_score_overtime'] = 0
    entry['home_team_score_overtime'] = 0


# TODO - How to get OT's? Try one
print(entry)
linescore_df.head()


{'away_team': 'Green Bay Packers', 'away_team_score_q1': 0, 'away_team_score_q2': 16, 'away_team_score_q3': 3, 'away_team_score_q4': 3, 'away_team_score_final': 25, 'home_team': 'Cincinnati Bengals', 'home_team_score_q1': 7, 'home_team_score_q2': 7, 'home_team_score_q3': 0, 'home_team_score_q4': 8, 'home_team_score_final': 22, 'away_team_score_overtime': 3, 'home_team_score_overtime': 0}


,Unnamed: 0,Unnamed: 1,1,2,3,4,OT,Final
0,via Sports Logos.net About logos,Green Bay Packers,0,16,3,3,3,25
1,via Sports Logos.net About logos,Cincinnati Bengals,7,7,0,8,0,22


In [180]:
tester_games

['/boxscores/202110070sea.htm',
 '/boxscores/202110100atl.htm',
 '/boxscores/202110100car.htm',
 '/boxscores/202110100cin.htm',
 '/boxscores/202110100htx.htm',
 '/boxscores/202110100jax.htm',
 '/boxscores/202110100min.htm',
 '/boxscores/202110100pit.htm',
 '/boxscores/202110100tam.htm',
 '/boxscores/202110100was.htm',
 '/boxscores/202110100rai.htm',
 '/boxscores/202110100sdg.htm',
 '/boxscores/202110100crd.htm',
 '/boxscores/202110100dal.htm',
 '/boxscores/202110100kan.htm',
 '/boxscores/202110110rav.htm']

In [106]:
# load in weeks_lkp into a new var to test that the saving of the lookup table worked
with open('tmp_vars/season_week_lkp_dict.pkl', 'rb') as f:
    lookup_dict_loaded = pickle.load(f)

In [142]:
### DON'T NEED BELOW CODE - DON'T NEED TO RENDER JS AT THIS POINT ###


# # Trying to Render Javascript using Selenium
# tester_game_next = weeks_lkp[1970][2][3][0]
# url_tester_to_get = base_url + tester_game_next

# # Selenium stuff:
# _options = Options()
# _options.headless = True
# driver = webdriver.Chrome(
#         executable_path="/Users/Jared/dev_work/chromedriver",
#         options=_options
#     )
# driver.get(url_tester_to_get)
# driver.execute_script("window.scrollTo(1,7000)")
# time.sleep(5)
# tester_source_code = driver.execute_script("return document.body.innerHTML;")

# # save the rendered html
# game_id = tester_game_next[tester_game_next.find(tester_game_next.split('/')[2]):tester_game_next.find('.htm')]
# with open(f'{game_save_location}/{game_id}.html', 'w+') as f:
#     f.write(tester_source_code)
# driver.close()

/var/folders/lx/v3kb98kn0mddnx06drz51sfh0000gn/T/ipykernel_35130/475831319.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(
